In [ ]:
from datetime import datetime
import yfinance as yf
import pandas as pd
import math
import statistics
import requests
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
import urllib.parse
import warnings
warnings.filterwarnings("ignore")

# Using API

In [ ]:
# https://stackoverflow.com/questions/63650430/python-requests-401-error-but-url-opens-in-browser

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}
base_url = 'https://www.nseindia.com/api/historical/foCPV?'
params = {
    "from":"11-03-2024",
    "to":"28-03-2024",
    "instrumentType":"OPTIDX",
    "symbol":"BANKNIFTY",
    "year":"2024",
    "expiryDate":"28-Mar-2024",
    "optionType":"CE",
    "csv":"true"
    }
full_url = base_url + urllib.parse.urlencode(params)

with requests.session() as s:
    # load cookies:
    s.get('https://www.nseindia.com/report-detail/fo_eq_security', headers=headers)
    # get data:
    ce_data = s.get(full_url, headers=headers).json()

call_options_chain = pd.DataFrame.from_dict(ce_data['data'])

params['optionType'] = "PE"
full_url = base_url + urllib.parse.urlencode(params)

with requests.session() as s:
    # load cookies:
    s.get('https://www.nseindia.com/report-detail/fo_eq_security', headers=headers)
    # get data:
    pe_data = s.get(full_url, headers=headers).json()

put_options_chain = pd.DataFrame.from_dict(pe_data['data'])

combined_options_chain = pd.concat([call_options_chain, put_options_chain]).sort_values(by=['FH_TIMESTAMP','FH_OPTION_TYPE']).reset_index(drop = True)

options = {"CE":{
                "S":[49500]
                },
           "PE":{
                "S":[43800]
                }
           }

cols = ["date"]
duplicates = []
for x in options:
  for y in options[x]:
    for z in options[x][y]:
      if (x + "_" + y + "_" + str(z)) in cols:
        count = cols.count(x + "_" + y + "_" + str(z))
        cols.append(x + "_" + y + "_" + str(z) + "_" + str(count+1))
        duplicates.append(x + "_" + y + "_" + str(z) + "_" + str(count+1))
      else:
        cols.append(x + "_" + y + "_" + str(z))

selected_options_dict = {}
for x in cols:
  selected_options_dict[x] = []

for x in combined_options_chain['FH_TIMESTAMP'].unique():
  if len(selected_options_dict['date']) == 0:
    selected_options_dict['date'].append(x)
  selected_options_dict['date'].append(x)

  for y in combined_options_chain.index:
    if combined_options_chain['FH_TIMESTAMP'][y] == x:

      if int(float(combined_options_chain['FH_STRIKE_PRICE'][y])) in options['CE']['S'] and combined_options_chain['FH_OPTION_TYPE'][y] == "CE":
        if len(selected_options_dict["CE_S_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))]) == 0:
          selected_options_dict["CE_S_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))].append(float(combined_options_chain['FH_OPENING_PRICE'][y]))
          selected_options_dict["CE_S_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))].append(float(combined_options_chain['FH_CLOSING_PRICE'][y]))
        else:
          selected_options_dict["CE_S_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))].append(float(combined_options_chain['FH_CLOSING_PRICE'][y]))

      if int(float(combined_options_chain['FH_STRIKE_PRICE'][y])) in options['CE']['B'] and combined_options_chain['FH_OPTION_TYPE'][y] == "CE":
        if len(selected_options_dict["CE_B_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))]) == 0:
          selected_options_dict["CE_B_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))].append(float(combined_options_chain['FH_OPENING_PRICE'][y]))
          selected_options_dict["CE_B_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))].append(float(combined_options_chain['FH_CLOSING_PRICE'][y]))
        else:
          selected_options_dict["CE_B_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))].append(float(combined_options_chain['FH_CLOSING_PRICE'][y]))

      if int(float(combined_options_chain['FH_STRIKE_PRICE'][y])) in options['PE']['B'] and combined_options_chain['FH_OPTION_TYPE'][y] == "PE":
        if len(selected_options_dict["PE_B_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))]) == 0:
          selected_options_dict["PE_B_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))].append(float(combined_options_chain['FH_OPENING_PRICE'][y]))
          selected_options_dict["PE_B_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))].append(float(combined_options_chain['FH_CLOSING_PRICE'][y]))
        else:
          selected_options_dict["PE_B_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))].append(float(combined_options_chain['FH_CLOSING_PRICE'][y]))

      if int(float(combined_options_chain['FH_STRIKE_PRICE'][y])) in options['PE']['S'] and combined_options_chain['FH_OPTION_TYPE'][y] == "PE":
        if len(selected_options_dict["PE_S_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))]) == 0:
          selected_options_dict["PE_S_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))].append(float(combined_options_chain['FH_OPENING_PRICE'][y]))
          selected_options_dict["PE_S_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))].append(float(combined_options_chain['FH_CLOSING_PRICE'][y]))
        else:
          selected_options_dict["PE_S_" + str(int(float(combined_options_chain['FH_STRIKE_PRICE'][y])))].append(float(combined_options_chain['FH_CLOSING_PRICE'][y]))

for x in duplicates:
  temp = "_".join(x.split("_")[0:3])
  selected_options_dict[x] = selected_options_dict[temp]

selected_options_data = pd.DataFrame.from_dict(selected_options_dict)

selected_options_data

,date,CE_S_22400,CE_B_22700,PE_S_22200,PE_S_21950,PE_B_21900,PE_B_22050
0,11-Mar-2024,375.20,182.90,121.00,76.55,63.55,94.95
1,11-Mar-2024,267.35,131.30,164.10,96.75,87.55,120.45
2,12-Mar-2024,264.65,124.90,142.70,80.50,72.15,101.55
3,13-Mar-2024,119.65,50.50,298.70,182.80,165.35,227.55
4,14-Mar-2024,154.75,57.00,189.25,106.45,93.15,133.40
5,15-Mar-2024,100.45,32.80,241.70,135.35,120.00,173.90
6,18-Mar-2024,85.70,24.75,229.70,122.70,108.00,159.45
7,19-Mar-2024,31.20,8.85,379.35,217.10,191.50,274.85
8,20-Mar-2024,19.80,4.40,340.25,180.40,157.60,236.95
9,21-Mar-2024,28.50,3.75,180.00,71.70,59.10,104.90


In [ ]:
pnl = [0]
for x in selected_options_data.index:
  if x != 0:
    temp = 0
    for y in selected_options_data.columns:
      if len(y.split("_")) > 1:
        if y.split("_")[0] == "CE":
          if y.split("_")[1] == "B":
            temp = temp + (float(selected_options_data[y][x]) - float(selected_options_data[y][x-1]))
          if y.split("_")[1] == "S":
            temp = temp + (float(selected_options_data[y][x-1]) - float(selected_options_data[y][x]))
        if y.split("_")[0] == "PE":
          if y.split("_")[1] == "S":
            temp = temp + (float(selected_options_data[y][x-1]) - float(selected_options_data[y][x]))
          if y.split("_")[1] == "B":
            temp = temp + (float(selected_options_data[y][x]) - float(selected_options_data[y][x-1]))
    pnl.append(temp)

selected_options_data["PnL"] = pnl

selected_options_data

,date,CE_S_22400,CE_B_22700,PE_S_22200,PE_S_21950,PE_B_21900,PE_B_22050,PnL
0,11-Mar-2024,375.20,182.90,121.00,76.55,63.55,94.95,0.00
1,11-Mar-2024,267.35,131.30,164.10,96.75,87.55,120.45,42.45
2,12-Mar-2024,264.65,124.90,142.70,80.50,72.15,101.55,-0.35
3,13-Mar-2024,119.65,50.50,298.70,182.80,165.35,227.55,31.50
4,14-Mar-2024,154.75,57.00,189.25,106.45,93.15,133.40,-9.15
5,15-Mar-2024,100.45,32.80,241.70,135.35,120.00,173.90,16.10
6,18-Mar-2024,85.70,24.75,229.70,122.70,108.00,159.45,4.90
7,19-Mar-2024,31.20,8.85,379.35,217.10,191.50,274.85,-6.55
8,20-Mar-2024,19.80,4.40,340.25,180.40,157.60,236.95,10.95
9,21-Mar-2024,28.50,3.75,180.00,71.70,59.10,104.90,29.05


In [ ]:
start_date = datetime(2024,3,11)
end_date = datetime(2024,3,12)

# spot_nifty_on_start = yf.Ticker("^NSEI").history(start=start_date, end=end_date)["Open"].to_list()[0]
spot_nifty_on_start = combined_options_chain['FH_UNDERLYING_VALUE'][0] #Underlying value as on the first day in the option chain

investment_amt = 1000000
lot_size = 50
margin_percent = 0.09

buy_opening_total = 0
no_of_sell_options = 0
for x in selected_options_data.columns:
  if len(x.split("_")) > 1:
    if x.split("_")[1] == "B":
      buy_opening_total = buy_opening_total + float(selected_options_data[x][0])
    else:
      no_of_sell_options = no_of_sell_options + 1

no_of_contract = math.floor(investment_amt / (buy_opening_total * lot_size + spot_nifty_on_start * lot_size * margin_percent * no_of_sell_options))

capital_deployed = buy_opening_total * lot_size * no_of_contract + spot_nifty_on_start * lot_size * margin_percent * no_of_sell_options * no_of_contract

net_pnl = selected_options_data["PnL"].sum()
total_pnl = net_pnl * lot_size * no_of_contract

print("Underlying Value of NIFTY on first day = ",spot_nifty_on_start)
print("Investment Amount = ",investment_amt)
print("Number of Contracts = ",no_of_contract)
print("Lot Size = ",lot_size)
print("Margin Percent for Selling Options = ",margin_percent*100,"%")
print("Capital Deployed = ",capital_deployed)
print("Capital Unused = ",investment_amt - capital_deployed)
print("Total P&L = ",total_pnl)
print("Percentage P&L = ",total_pnl / capital_deployed *100, "%")
print("Number of Loss Days = ",(selected_options_data["PnL"] < 0).sum())
print("Number of Win Days = ",(selected_options_data["PnL"] > 0).sum())
print("Max Profit in 1 day = ",(selected_options_data["PnL"]).max() * lot_size * no_of_contract)
print("Max Loss in 1 day = ",(selected_options_data["PnL"]).min() * lot_size * no_of_contract)
print("Average P&L in 1 day = ",(selected_options_data["PnL"]).mean() * lot_size * no_of_contract)
print("Standard Deviation 1 day = ",(selected_options_data["PnL"]).std() * lot_size * no_of_contract)

Underlying Value of NIFTY on first day =  22332.65
Investment Amount =  1000000
Number of Contracts =  3
Lot Size =  50
Margin Percent for Selling Options =  9.0 %
Capital Deployed =  955682.3250000001
Capital Unused =  44317.67499999993
Total P&L =  33000.0
Percentage P&L =  3.453030273422709 %
Number of Loss Days =  4
Number of Win Days =  9
Max Profit in 1 day =  7327.500000000002
Max Loss in 1 day =  -1372.500000000001
Average P&L in 1 day =  2357.142857142857
Standard Deviation 1 day =  2810.3098065500194


In [ ]:
payoff_chart_data = selected_options_data.copy(deep=True)

for x in payoff_chart_data:
  if len(x.split("_")) > 1:
    if x.split("_")[1] == "B":
      payoff_chart_data[x] = payoff_chart_data[x] * - 1

net_prem = payoff_chart_data.iloc[0,1:-1].sum()

payoffs = {"spot" : [], "net_premium": []}
for x in payoff_chart_data:
  if len(x.split("_")) > 1:
    payoffs[x] = []
payoffs['net_payoff'] = []

for x in range(21500, 23500, 5):
  payoffs['spot'].append(x)
  payoffs['net_premium'].append(net_prem)
  temp = net_prem
  for y in payoff_chart_data:
    if len(y.split("_")) > 1:

      if y.split("_")[0] == "CE" and y.split("_")[1] == "S":
        if int(y.split("_")[2]) > x:
          payoffs[y].append(0)
        else:
          payoffs[y].append((x - int(y.split("_")[2])) * -1)
          temp = temp + ((x - int(y.split("_")[2])) * -1)

      if y.split("_")[0] == "CE" and y.split("_")[1] == "B":
        if int(y.split("_")[2]) > x:
          payoffs[y].append(0)
        else:
          payoffs[y].append(x - int(y.split("_")[2]))
          temp = temp + (x - int(y.split("_")[2]))

      if y.split("_")[0] == "PE" and y.split("_")[1] == "S":
        if int(y.split("_")[2]) < x:
          payoffs[y].append(0)
        else:
          payoffs[y].append((int(y.split("_")[2]) - x) * -1)
          temp = temp + ((int(y.split("_")[2]) - x) * -1)

      if y.split("_")[0] == "PE" and y.split("_")[1] == "B":
        if int(y.split("_")[2]) < x:
          payoffs[y].append(0)
        else:
          payoffs[y].append(int(y.split("_")[2]) - x)
          temp = temp + (int(y.split("_")[2]) - x)

  payoffs['net_payoff'].append(temp)

payoffs_data = pd.DataFrame.from_dict(payoffs)
payoffs_data

,spot,net_premium,CE_S_22400,CE_B_22700,PE_S_22200,PE_S_21950,PE_B_21900,PE_B_22050,net_payoff
0,21500,231.35,0,0,-700,-450,400,550,31.35
1,21505,231.35,0,0,-695,-445,395,545,31.35
2,21510,231.35,0,0,-690,-440,390,540,31.35
3,21515,231.35,0,0,-685,-435,385,535,31.35
4,21520,231.35,0,0,-680,-430,380,530,31.35
...,...,...,...,...,...,...,...,...,...
395,23475,231.35,-1075,775,0,0,0,0,-68.65
396,23480,231.35,-1080,780,0,0,0,0,-68.65
397,23485,231.35,-1085,785,0,0,0,0,-68.65
398,23490,231.35,-1090,790,0,0,0,0,-68.65


In [ ]:
import plotly.express as px

fig = px.scatter(payoffs_data, x='spot', y='net_payoff', title="Payoff Chart", color='net_payoff', color_continuous_scale=[[0, 'red'], [0.5, 'orange'], [1, 'green']]).update_layout(
    xaxis_title="Spot Price on Expiry", yaxis_title="Net Payoff",  font_size=16, plot_bgcolor='white'
)
fig.update_xaxes({'gridcolor': 'white', 'zerolinecolor': 'blue', 'linecolor': 'blue'},
    ticks='outside',
    showline=True,
    linecolor='black')
fig.update_yaxes({'gridcolor': 'white', 'zerolinecolor': 'blue'},ticks='outside',
    showline=True,
    linecolor='black')
fig.show()